## 准备数据，从文本中构建向量 ##

In [1]:
#词到向量的转换函数
def loadDataSet():
    postingList=[
        ['my','dog','has','flea','problem','help','please'],
        ['maybe','not','take','him','to','dog','park','stupid'],
        ['my','dalmation','is','so','cute','I','love','him'],
        ['stop','posting','stupid','worthless','grabage'],
        ['mr','licks','ate','my','steak','how','to','stop','him'],
        ['quit','buyiing','worthless','dog','food','stupid']
    ]
    classVec=[0,1,0,1,0,1]#1表示侮辱性文字 0表示正常文字
    return postingList,classVec
#生成不重复的单词
def createVocabList(dataSet):
    vocabSet=set([])
    #合并
    for document in dataSet:
        vocabSet=vocabSet | set(document)
    return list(vocabSet)
def setOfWords2Vec(vocabList,inputSet):
    returnVec=[0]*len(vocabList)#生成全是0 长度为字典长度的字典
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)]=1
        else:
            print('the word %s is not in my vocabulary'%(word))
    return returnVec
listOPosts,listClasses=loadDataSet()
myVocabList=createVocabList(listOPosts)
print(myVocabList)

['him', 'so', 'posting', 'stop', 'ate', 'flea', 'to', 'quit', 'food', 'park', 'my', 'dog', 'not', 'cute', 'has', 'worthless', 'is', 'licks', 'maybe', 'buyiing', 'mr', 'help', 'how', 'I', 'dalmation', 'take', 'please', 'steak', 'grabage', 'stupid', 'problem', 'love']


In [2]:
setOfWords2Vec(myVocabList,listOPosts[0])

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0]

## 训练算法，从词向量到计算概率 ##

In [3]:
from numpy import *
#朴素贝叶斯分类训练函数
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs=len(trainMatrix)
    numWords=len(trainMatrix[0])
    pAbusive=sum(trainCategory)/float(numTrainDocs)
    p0Num=zeros(numWords)
    p1Num=zeros(numWords)
    p0Denom=0.0
    p1Denom=0.0
    for i in range(numTrainDocs):
        if trainCategory[i]==1:
            p1Num+=trainMatrix[i]
            p1Denom+=sum(trainMatrix[i])
        else:
            p0Num+=trainMatrix[i]
            p0Denom+=sum(trainMatrix[i])
    p1Vect=p1Num/p1Denom
    p0Vect=p0Num/p0Denom
    return p0Vect,p1Vect,pAbusive

In [4]:
trainMat=[]
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
p0V,p1V,pAb=trainNB0(trainMat,listClasses)
print(pAb)
print(p0V)
print(p1V)

0.5
[0.08333333 0.04166667 0.         0.04166667 0.04166667 0.04166667
 0.04166667 0.         0.         0.         0.125      0.04166667
 0.         0.04166667 0.04166667 0.         0.04166667 0.04166667
 0.         0.         0.04166667 0.04166667 0.04166667 0.04166667
 0.04166667 0.         0.04166667 0.04166667 0.         0.
 0.04166667 0.04166667]
[0.05263158 0.         0.05263158 0.05263158 0.         0.
 0.05263158 0.05263158 0.05263158 0.05263158 0.         0.10526316
 0.05263158 0.         0.         0.10526316 0.         0.
 0.05263158 0.05263158 0.         0.         0.         0.
 0.         0.05263158 0.         0.         0.05263158 0.15789474
 0.         0.        ]


## 测试算法：根据现实情况修改分类器 ##

### 1.极大似然估计，出现0抹去的情况，使用拉普拉斯修正 ###
### 2.连续相乘导致下溢 ###

In [5]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs=len(trainMatrix)
    numWords=len(trainMatrix[0])
    pAbusive=sum(trainCategory)/float(numTrainDocs)
    #使用拉普拉斯修正
    p0Num=ones(numWords)
    p1Num=ones(numWords)
    p0Denom=2.0
    p1Denom=2.0
    for i in range(numTrainDocs):
        if trainCategory[i]==1:
            p1Num+=trainMatrix[i]
            p1Denom+=sum(trainMatrix[i])
        else:
            p0Num+=trainMatrix[i]
            p0Denom+=sum(trainMatrix[i])
    #使用对数防止下溢
    p1Vect=log(p1Num/p1Denom)
    p0Vect=log(p0Num/p0Denom)
    return p0Vect,p1Vect,pAbusive

## 朴素贝叶斯分类函数 ##

In [6]:
def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):
    p1=sum(vec2Classify*p1Vec)+log(pClass1)
    p0=sum(vec2Classify*p0Vec)+log(1.0-pClass1)
    if p1>p0:
        return 1
    else:
        return 0
def testingNB():
    listOPosts,listClasses=loadDataSet()
    myVocabList=createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
    p0V,p1V,pAb=trainNB0(trainMat,listClasses)
    testEntry=['love','my','dalmation']
    thisDoc=array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classifyed as : ',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry=['stupid','garbage']
    thisDoc=array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as : ',classifyNB(thisDoc,p0V,p1V,pAb))
testingNB()

['love', 'my', 'dalmation'] classifyed as :  0
the word garbage is not in my vocabulary
['stupid', 'garbage'] classified as :  1


##  朴素贝叶斯词袋模型 ##

In [7]:
def bagOfWords2VecMN(vocabList,inputSet):
    returnVec=[0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)]+=1
    return returnVec

# 示例：使用朴素贝叶斯过滤垃圾邮件 #

In [8]:
mySent='this book is the best book on python or M.L i have ever laid eyes upon'
mySent.split()


['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'M.L',
 'i',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon']

可以看到，切分效果不错，但是标点符号也被当成了词的一部分，可以用正则表示来切分句子，其中分隔符是除单词，数字外的任意字符串

In [9]:
import re
regEx=re.compile('\\W*')
listOfTokens=regEx.split(mySent)
listOfTokens

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: split() requires a non-empty pattern match.
  This is separate from the ipykernel package so we can avoid doing imports until


['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'M',
 'L',
 'i',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon']

In [10]:
[tok.lower() for tok in listOfTokens if(len(tok)>0)]

['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'm',
 'l',
 'i',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon']

In [11]:
emailText=open('email/ham/6.txt').read()
listOfTokens=regEx.split(emailText)

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: split() requires a non-empty pattern match.
  


## 测试：使用朴素贝叶斯进行交叉验证 ##

In [14]:
def textParse(bigString):    #input is big string, #output is word list
    import re
    listOfTokens = re.split(r'\W*', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2] 
def spamTest():
    docList=[]; classList = []; fullText =[]
    for i in range(1,26):
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    trainingSet = list(range(50)); testSet=[]           #create test set
    for i in range(10):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])  
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
            print ("classification error",docList[docIndex])
    print ('the error rate is: ',float(errorCount)/len(testSet))
    #return vocabList,fullText
    

In [15]:
spamTest()

classification error ['yay', 'you', 'both', 'doing', 'fine', 'working', 'mba', 'design', 'strategy', 'cca', 'top', 'art', 'school', 'new', 'program', 'focusing', 'more', 'right', 'brained', 'creative', 'and', 'strategic', 'approach', 'management', 'the', 'way', 'done', 'today']
classification error ['yeah', 'ready', 'may', 'not', 'here', 'because', 'jar', 'jar', 'has', 'plane', 'tickets', 'germany', 'for']
the error rate is:  0.2


D:\anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


# 使用朴素贝叶斯分类器从个人广告 #

In [16]:
def calcMostFreq(vocabList,fullText):
    import operator
    freqDict = {}
    for token in vocabList:
        freqDict[token]=fullText.count(token)
    sortedFreq = sorted(freqDict.items(), key=operator.itemgetter(1), reverse=True) 
    return sortedFreq[:30]       

def localWords(feed1,feed0):
    import feedparser
    docList=[]; classList = []; fullText =[]
    print(len(feed1['entries']))
    print(len(feed0['entries']))
    minLen = min(len(feed1['entries']),len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1) #NY is class 1
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    top30Words = calcMostFreq(vocabList,fullText)   #remove top 30 words
    for pairW in top30Words:
        if pairW[0] in vocabList: vocabList.remove(pairW[0])
    trainingSet = list(range(2*minLen)); testSet=[]           #create test set
    for i in range(20):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])  
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is: ',float(errorCount)/len(testSet))
    return vocabList,p0V,p1V

In [17]:
import feedparser
# ny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')
# sf = feedparser.parse('http://sfbay.craigslist.org/stp/index.rss')
ny = feedparser.parse('http://www.nasa.gov/rss/dyn/image_of_the_day.rss')
sf = feedparser.parse('http://rss.yule.sohu.com/rss/yuletoutiao.xml')
vocabList, pSF, pNY = localWords(ny, sf)

60
30
the error rate is:  0.45


D:\anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [18]:
def getTopWords(ny,sf):
    import operator
    vocabList,p0V,p1V=localWords(ny,sf)
    topNY=[]; topSF=[]
    for i in range(len(p0V)):
        if p0V[i] > -6.0 : topSF.append((vocabList[i],p0V[i]))
        if p1V[i] > -6.0 : topNY.append((vocabList[i],p1V[i]))
    sortedSF = sorted(topSF, key=lambda pair: pair[1], reverse=True)
    print("SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**")
    for item in sortedSF:
        print(item[0])
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print("NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**")
    for item in sortedNY:
        print(item[0])

In [19]:

getTopWords(ny, sf)

60
30
the error rate is:  0.5
SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**
据香港媒体报道
小海绵
据台湾媒体报道
假张译微信招表演学生
我不是潘金莲
有网友拍到白百何带儿子现身上海大悦城
高晓松在京出席某地图发布会
胡海泉否认羽泉解散
他果断否认羽泉解散
购入浅水湾道低密度住宅怡峰一个望海景单位
由范冰冰主演
张译发文辟谣做导演教学生
现场他被聘为首席采集娱乐官
白百何头戴鸭舌帽
乒乓王子
突然被莫明伤害到的人物应该给他很多空间
孔令辉的国家女乒帅位已经由同僚李隼代替
根据孩子的背影与身
张译曝光骗子以他名义冒充副导演和网友聊天截
伊能静在文中解释道
只是角度选择的问题
艺人杨颖
一直都专心照顾孩子
秦昊微博截图
被送到医院抢救仍宣告不治
乔欣说自己还有个姐姐
跨界上演了一部
被问及近期陈羽凡因婚变宣布无限期离开娱乐圈一事
他笑说
直挺挺地倒下
前一秒还拿麦克风唱着歌
高育良
张凯丽在微博晒和
做什么事儿
angelababy
金沙的代表律师已经向香港高等法院提出终止法律程序
海外在娱乐方面的投资是赚钱
就该脚踏实地的努
被怀疑从头到尾都是自导自演
张智霖谈做郭富城伴郎感受
我不是一个电影制片人
羽泉当然存在啊
由后者带领中国女队出战世界乒乓球锦标赛
潘金莲是不正经的女
中国女乒队前主教练
活地图
全剧最期待的对手戏
艺人张智霖
他每年只和母亲一起去看电影一次
身穿白色上衣
向来孝顺的谢霆锋今年的父亲节礼物特别大份
孔令辉的欠债风波又有新进展
我们每
他倒下的瞬间影片在25日曝光
高晓松发言
并首度自曝家庭情况
谈到对于二胎的看法
和她们合照压力好大
我们要给在乎的人空间和包容
西里_cili
需要理解
问他到那他的太太袁咏仪呢
简直就是中国版的无间道夫妇
4月20日
一旁的乔欣连忙接话
冯小刚2016年上映的电影
有网友爆料白百何携子现身上海大悦城
4月17日晚
人民的名义
4月18日晚
伊能静在微博发长文为老公秦昊在节目中被黑喊冤
实用尺价约4
资深港星曾守明23日在广州表演
采访时认为
张凯丽在微
唐禹哲发文否认恋网红
金融时报
25日
但女方随后遭到网友起底
38万港币
chilam
一名助理指出
胡海泉在参加活动时
未料唱歌唱到一半突然昏倒


D:\anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
